# Lab 6 

Ali Tejani, amt3639

Caroline Yao, chy253

Allen Hwang, ah45755

## Problem 1

### Estimating beta with linear regression

In [481]:
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge

In [482]:
n = 51
p = 50

In [483]:
def estimate_b(reg,n,p,seed=None):
    np.random.seed(seed=seed)
    b_hat = []
    for _ in range(100):
        X = np.random.normal(0,1,(n,p))
        e = np.random.normal(0,.25,(n,1))
        b = np.ones((p,1))
        y = np.dot(X,b) + e
        reg.fit(X,y)
        reg.coef_
        b_hat.append(reg.coef_)
    b_hat = np.array(b_hat)
    print b_hat.mean(axis=0).mean()
    print b_hat.var(axis=0).mean()

In [484]:
estimate_b(LinearRegression(fit_intercept=False),n,p,seed=1)

1.00655839826
0.227150750588


In [485]:
alphas = [0.01,0.1,1,10,100]
for a in alphas:
    estimate_b(Ridge(alpha=a),n,p,seed=1)

0.982576226953
0.0641155927754
0.950031875305
0.0392228415033
0.863646895296
0.0623827708922
0.642854600564
0.091449034224
0.268810522308
0.0409478695197


## Problem 2: Chapter 6, Problem 9

### a.  Split the data set into a training set and a test set

In [486]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
np.random.seed()

In [487]:
college = pd.read_csv('College.csv',index_col=0)
college['Private'] = college['Private'].replace('Yes',1).replace('No',0)
collegeX = college[[i for i in college.axes[1] if i != 'Apps']]
collegeY = college['Apps']
college_trainX,college_testX,college_trainY,college_testY = train_test_split(collegeX,collegeY)

### b. Fit a linear model using least squares on the training set, and report the test error obtained

In [488]:
def reg_score(reg):
    reg.fit(college_trainX,college_trainY)
    print reg.score(college_testX,college_testY)

In [489]:
reg_score(LinearRegression())

0.91907395571


### c.  Fit a ridge regression model on the training set, with λ chosen by cross-validation. Report the test error obtained.

In [490]:
from sklearn.linear_model import RidgeCV,LassoCV
ridge_reg = RidgeCV(alphas=[0.001,0.005,0.01,0.05,0.1,0.5,1,10,100])
reg_score(ridge_reg)
ridge_reg.alpha_

0.917704186648


100.0

### d.  Fit a lasso model on the training set, with λ chosen by crossvalidation. Report the test error obtained, along with the number of non-zero coefficient estimates.

In [491]:
lasso_reg = LassoCV(alphas=[0.0001,0.001,0.005,0.01,0.05,0.1,0.5,1,10,100])
reg_score(lasso_reg)
print lasso_reg.alpha_
print lasso_reg.coef_

0.917040121118
100.0
[ -0.00000000e+00   1.64025416e+00  -7.96964269e-01   3.80796460e+01
  -7.75219978e+00   4.15798888e-02   8.15921970e-02  -9.94322890e-02
   7.91305456e-02  -1.25698902e-01   2.90662004e-03  -6.18744115e+00
  -4.23860922e+00   2.32248809e+01   1.39469676e+00   1.07631842e-01
   6.55439108e+00]


### e. Fit a PCR model on the training set, with M chosen by cross-validation. Report the test error obtained, along with the value of M selected by cross-validation

In [492]:
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
pca = PCA()
linreg = LinearRegression()
pipe = Pipeline(steps=[('pca',pca),('linear',linreg)])

In [493]:
gscv = GridSearchCV(pipe, dict(pca__n_components=np.arange(1,college_trainX.shape[1]+1)),cv=10)
gscv.fit(college_trainX,college_trainY)
print gscv.score(college_testX,college_testY)
print gscv.best_params_

0.91907395571
{'pca__n_components': 17}


### f. Fit a PLS model on the training set, with M chosen by crossvalidation. Report the test error obtained, along with the value of M selected by cross-validation.

In [494]:
from sklearn.cross_decomposition import PLSRegression

In [497]:
from sklearn.preprocessing import scale 
pls = PLSRegression()
# pls.fit(scale(college_trainX),college_trainY)
# pls.score(scale(college_testX),college_testY)
gscv = GridSearchCV(pls, dict(n_components=np.arange(1,college_trainX.shape[1]+1)),cv=10)
gscv.fit(scale(college_trainX),college_trainY)
print gscv.score(scale(college_testX),college_testY)
print gscv.best_score_
gscv.cv_results_

0.909702851329
0.908829712601


{'mean_fit_time': array([ 0.00349998,  0.00200002,  0.00449998,  0.0016    ,  0.005     ,
         0.0049    ,  0.0062    ,  0.00339999,  0.0036    ,  0.00470002,
         0.0049    ,  0.00339999,  0.00780001,  0.0049    ,  0.00480001,
         0.00670002,  0.005     ]),
 'mean_score_time': array([  6.99996948e-04,   2.99978256e-04,   3.00025940e-04,
          0.00000000e+00,   3.99994850e-04,   9.99927521e-05,
          1.60000324e-03,   1.70001984e-03,   0.00000000e+00,
          0.00000000e+00,   9.99927521e-05,   1.60000324e-03,
          0.00000000e+00,   9.99927521e-05,   0.00000000e+00,
          0.00000000e+00,   1.60000324e-03]),
 'mean_test_score': array([ 0.7669108 ,  0.83769125,  0.86952916,  0.88135124,  0.89804753,
         0.90591822,  0.90708938,  0.90832091,  0.90854485,  0.90858227,
         0.90882971,  0.90867926,  0.90874952,  0.90879676,  0.90879438,
         0.90881104,  0.9088109 ]),
 'mean_train_score': array([ 0.76620373,  0.84184187,  0.87336976,  0.90577346,